# Segmenting and Clustering Neighborhoods in Toronto - Assignment

Retreving the Wikipedia page using Request lib & importing the html lib to read the table data

In [1]:
import requests 
url1='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
import lxml.html as lh
page = requests.get(url1)
doc = lh.fromstring(page.content)
tr_elements = doc.xpath('//tr')


Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20181101000029-0000


Reteriving the columns & values of the table using the html Tags

In [2]:
col=[]
i=0
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print ('%d:"%s"'%(i,name))
    col.append((name,[]))


1:"Postcode"
2:"Borough"
3:"Neighbourhood
"


In [3]:
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    if len(T)!=3:
        break
    i=0
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
            try:
                data=int(data)
            except:
                pass
        col[i][1].append(data)
        i+=1        
[len(C) for (title,C) in col]

[289, 289, 289]

Using Pandas to convert the data into a table format

In [4]:
import pandas as pd
My_table={title:column for (title,column) in col}
df=pd.DataFrame(My_table)
df.rename(columns={'Neighbourhood\n': 'Neighbourhood'}, inplace=True)
df.columns

df=df[['Postcode','Borough','Neighbourhood']]



In [5]:
df.replace('\n','', regex=True, inplace=True)
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Dropped the rows that contain 'Not Assigned'

In [6]:
df=df[~df['Neighbourhood'].isin(['Not assigned'])]
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


Appiled the Groupby function to combine the Neighbourhood -

In [7]:
df = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df[['Postcode','Borough','Neighbourhood']].head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Importing the Latitude & Longitud codes from the Geospatial_data.csv

In [10]:
url_geo='http://cocl.us/Geospatial_data'
geo_df = pd.read_csv(url_geo)
geo_df.head()



,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [16]:
geo_df.rename(columns={'Postal Code': 'Postcode'}, inplace=True)


In [17]:
geo_df.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merging the two tables based on the Postal codes.

In [20]:
df_new = pd.merge(df, geo_df, on='Postcode')
df_new.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
